<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/LLM/training/examples_fine_tune_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downgrade protobuf to a compatible version - otherwise save_pretrained_gguf fails on google colab
!pip install "protobuf>=3.19.0,<4.0.0" --quiet
# also for google colab
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:

!pip install unsloth --quiet
# !pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes transformers datasets --quiet



In [ ]:
from google.colab import userdata
import os
os.environ['WANDB_API_KEY']=userdata.get('WANDB_API_KEY')


Load the basemodel in unsloth

In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Adjust based on your data length
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

Add LoRA Adapter to the layers for efficient training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank - higher = more parameters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

The training dataset that we read in looks like:

[
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
]

In [ ]:
!wget --no-cache https://raw.githubusercontent.com/ebamberg/research-projects-ml/refs/heads/main/LLM/training/logfile_data_synthetic_train.json -O training_data.json
!wget --no-cache https://raw.githubusercontent.com/ebamberg/research-projects-ml/refs/heads/main/LLM/training/logfile_data_synthetic_eval.json -O eval_data.json

In [ ]:
import json
from datasets import Dataset

with open("training_data.json", "r") as f:
    data = json.load(f)
# Format for training
def format_chat_template(item):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": item['input']},
            {"role": "assistant", "content": item['output']}
        ],
        tokenize=False,
        add_generation_prompt=False
    )
# Create the training dataset
formatted_data = [{"text": format_chat_template(item)} for item in data]
dataset = Dataset.from_list(formatted_data)
# Check what it looks like
print("Sample training example:")
print(formatted_data[0]["text"])

We are ready to go, so start the training !

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

training_steps=60

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=1, # Reduced batch size
        gradient_accumulation_steps=8, # Increased accumulation steps to maintain similar effective batch size
        warmup_steps=5,
        max_steps=training_steps,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",  # Use "adamw_torch" if you get optimizer errors
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=30,
    ),
)
# Start training! 🚀
trainer.train()

Evaluate our model

expected output: Suspicious, Suspious, Nornal

In [ ]:
eval_prompts = ["classify the following log event sequence as normal or suspicious\n91.189.89.199,sess_752130,root,2025-07-27 14:22:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:24:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:26:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:28:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:30:26,Authentication failed for user root,ERROR\n91.189.89.199,sess_752130,root,2025-07-27 14:32:26,Authentication failed for user root,ERROR\n91.189.89.199,sess_752130,root,2025-07-27 14:34:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:36:26,Authentication failed for user root,ERROR\n91.189.89.199,sess_752130,root,2025-07-27 14:42:26,Brute force attack identified,ERROR",
                "classify the following log event sequence as normal or suspicious\n77.88.55.80,sess_256163,emma.taylor,2025-07-30 14:22:26,User authentication successful,INFO\n77.88.55.80,sess_256163,emma.taylor,2025-07-30 14:24:26,Database query executed,INFO\n77.88.55.80,sess_256163,emma.taylor,2025-07-30 14:26:26,Abnormal data access volume,ERROR\n77.88.55.80,sess_256163,emma.taylor,2025-07-30 14:28:26,Data exfiltration pattern,ERROR\n77.88.55.80,sess_256163,emma.taylor,2025-07-30 14:30:26,Suspicious file access pattern,ERROR\n77.88.55.80,sess_256163,emma.taylor,2025-07-30 14:32:26,Network intrusion detected,ERROR",
                "classify the following log event sequence as normal or suspicious\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:22:26,User authentication successful,INFO\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:24:26,Form submission processed,INFO\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:24:26,Resource access granted,INFO\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:25:26,Session timeout warning,DEBUG\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:37:26,Session started,DEBUG\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:40:26,Session timeout warning,DEBUG\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 14:42:26,User logout successful,DEBUG\n192.168.1.10,sess_750740,kevin.lee,2025-07-24 15:02:26,User logout successful,INFO",
        ]
print(eval_prompts)

In [ ]:
# Switch to inference mode
FastLanguageModel.for_inference(model)

for test in eval_prompts:
  print(test)

  # Format input using chat template
  messages = [{"role": "user", "content": test}]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to("cuda")
  # Generate response
  with torch.no_grad():
      outputs = model.generate(
          input_ids=inputs,
          max_new_tokens=256,
          use_cache=True,
          temperature=0.1,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )
  # Extract just the new generated text (not the full conversation)
  response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  print("Model output:")
  print(response.strip())


SAVE our fine-tuned model in the GGUF format which is compatible to Ollama.

In [ ]:
# save_pretrained_gguf is a unsloth function. this is not available on standard hugging face models
model.save_pretrained_gguf(
    "fine_tuned_model",
    tokenizer,
    quantization_method="q4_k_m"  # Good balance of size/quality
)

In [ ]:
%%writefile ./fine_tuned_model/Modelfile.py
FROM ./unsloth.Q4_K_M.gguf
PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER stop ["<|endoftext|>"]
TEMPLATE "{{ .Prompt }}"
SYSTEM "You are a specialized devop trained to analyze log file."

In [ ]:
!zip -r /content/fine_tuned_model_log_file_analyser.zip /content/fine_tuned_model/

from google.colab import files
files.download("/content/fine_tuned_model_log_file_analyser.zip")

you can use ollama to create a model from the modelfile:

ollama create \<modelname\> -f Modelfile

In [ ]:
!pip install ollama --quiet


In [ ]:
!cd /content/fine_tuned_model/
get_ipython().system_raw("ollama create log_file_analyser -f Modelfile")